In [29]:
from keras.datasets import mnist
import numpy as np
import pandas as pd
import sys
import copy
import math
from sklearn.preprocessing import normalize

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape([60000,28*28])
x_train.shape

(60000, 784)

In [30]:
n_sample=300
X=x_train[:n_sample,:]
X.shape

(300, 784)

A partir de las celdas de abajo se definirá una variación de la función SpectralClustering de sklearn, para logar rescatar el *manifold* que contiene el ebedding inicial con *eigenvectors*

In [31]:
import numpy as np
import sys
import copy

class MatchingPursuit:
	"""Simple implementation of the Matching Pursuit (MP) algorithm
	
  FUENTE: https://github.com/mitscha/ssc_mps_py/blob/master/matchingpursuit.py
  
	Parameters
	----------
	
	smax: int
		Maximum number of MP iterations
	pmax: int, optional
		Maximum sparsity level of x (default: pmax = smax)
	tol: float, optional
		Threshold on approximation quality ||Ax - y|| (default: 0.0)
		
	Attributes
	----------
	
	coef_: array, shape (n_samples)
		coefficient vector (solution)
	
	
	Note: Stops after smax iterations, or when approximation quality specified by tol
	      is attained, or when the sparsity level of the coefficient vector is pmax
	
	"""
	
	def __init__(self,smax,pmax=None,tol=None):
		self._smax = smax if smax != None else sys.maxsize
		self._pmax = pmax if pmax != None else smax
		self._tol = 0.0 if tol == None else tol
		self.coef_ = None
	
	def fit(self,A,y):
		"""
		Finds a sparse (approximate) solution x to Ax = y
		
		Parameters
		----------
		
		X: dictionary, array, shape (n_features, n_samples)
		y: target, array, shape (n_features)
		
		"""
		
		assert(len(A.shape) == 2)
		assert(len(y.shape) == 1 and A.shape[0] == y.shape[0])
		
		x = np.zeros(A.shape[1])
		r = copy.deepcopy(y)
		nit = 0
		while np.linalg.norm(r) > self._tol \
			and nit < self._smax \
			and np.sum(np.abs(x) > 0) < self._pmax:
			
			idx = np.argmax(np.dot(r.T,A))
			dx = np.dot(r.T,A[:,idx])/np.dot(A[:,idx].T,A[:,idx])
			x[idx] += dx
			r -= dx*A[:,idx]
			nit += 1
		
		self.coef_ = x

In [1]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.cluster import SpectralClustering
from sklearn.manifold import SpectralEmbedding
from sklearn.preprocessing import normalize

def ssc_mps_modificado(X,smax,L,tol=None,alg_name='OMP',pmax=None):
	"""
  Esta es una versión modificada de la función 'ssc_mps', implementada por
  https://github.com/mitscha/ssc_mps_py/blob/master/ssc_mps.py
  
	Implements Sparse Subspace Clustering-Orthogonal Matching Pursuit (SSC-OMP) and 
	SSC-Matching Pursuit (SSC-MP)
	
	Parameters
	----------
	
	X: array, shape (n_samples, n_features)
		data matrix
	smax: int
		Maximum number of OMP/MP iterations
	L: int
		Number of clusters
	tol: float, optional
		Threshold on approximation quality
	alg_name: string, optional
		'OMP' (default) or 'MP'
	pmax:
		Maximum sparsity level for MP
	
	
	Note: 
	
	- Stopping behavior:
	  SSC-OMP: Stop after smax iterations if tol=None, stop when approximation quality
	           specified by tol is attained otherwise
	  SSC-MP:  Stop after smax iterations, or when approximation quality specified by tol
	           is attained, or when the sparsity level of the coefficient vector is pmax
	- See https://arxiv.org/abs/1612.03450 for a discussion of the stopping criteria
	
	"""
	
	
	XX = np.array(X).T
	XX = normalize(XX - np.outer(np.mean(XX, axis=1), np.ones(XX.shape[1])), axis=0)
    
	assert(len(XX.shape) == 2)
	
	m = XX.shape[0]
	N = XX.shape[1]
	
	
	alg = None
	if alg_name == 'MP':
		alg = MatchingPursuit(smax, pmax, tol)
	else:
		alg = OrthogonalMatchingPursuit(
			n_nonzero_coefs=smax, 
			tol=tol, 
			fit_intercept=False, 
			normalize=False)
	
	
	C = np.zeros((N,N))
	
	for i in range(N):
		data_idx = [j for j in range(i)]
		data_idx.extend([j for j in range(i+1,N)])
		alg.fit(XX[:,data_idx],np.squeeze(XX[:,i]))
		c = np.zeros(N)
		c[data_idx] = alg.coef_
		C[:,i] = c
	maps=SpectralEmbedding(affinity='precomputed', n_components=L, eigen_solver='arpack').fit(np.abs(C)+np.abs(C.T))
  # Se utiliza ahora una descomposición por eigenvectors
	sc = SpectralClustering(n_clusters=L, affinity='precomputed', n_init=50, n_jobs=-1, eigen_solver='arpack')
	sc.fit(np.abs(C) + np.abs(C.T)) # Se introduce la matriz de afinidad descrita en SUB-GAN
	
	return sc.labels_, maps.embedding_

In [26]:
K = 15 # Numero de grupos
soft_assign, eigen_embedding=ssc_mps_modificado(X,K,K)

In [27]:
# Verificamos que se obtienen las asignaciones correctas y el embedding tras la descomposición:
soft_assign.shape, eigen_embedding.shape

((300,), (300, 10))

In [28]:
# Este es el resultado del SpectralClustering utilizando la descomposición por
# eigenvectors con el número equivalente de dimensiones = 10
pd.crosstab(soft_assign,y_train[:n_sample]) 

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,0,0,0,18,0,3,0,0,6,1
1,0,0,0,0,0,0,0,15,0,0
2,0,0,0,0,0,13,0,0,0,0
3,5,0,0,0,0,0,21,0,0,0
4,7,10,15,7,17,4,5,4,12,7
5,21,0,0,0,1,0,2,0,0,0
6,0,8,12,9,2,3,1,2,2,2
7,0,18,0,0,0,0,0,0,0,0
8,1,0,1,0,12,0,0,8,1,21


In [35]:
soft_assign

array([4, 5, 8, 9, 8, 4, 6, 0, 6, 4, 4, 2, 6, 3, 7, 1, 8, 4, 3, 8, 6, 3,
       4, 9, 4, 6, 8, 0, 6, 4, 6, 4, 3, 4, 3, 2, 4, 3, 8, 3, 7, 0, 8, 8,
       4, 8, 4, 2, 6, 0, 4, 5, 1, 8, 8, 4, 5, 8, 8, 4, 5, 4, 3, 4, 4, 2,
       3, 4, 5, 5, 6, 1, 7, 3, 6, 4, 6, 4, 7, 1, 0, 4, 4, 3, 8, 0, 6, 4,
       5, 4, 3, 1, 8, 3, 4, 5, 8, 4, 4, 4, 6, 1, 7, 1, 6, 4, 4, 4, 5, 4,
       8, 0, 7, 4, 5, 4, 8, 6, 4, 3, 6, 5, 6, 1, 7, 0, 4, 4, 4, 3, 4, 4,
       4, 8, 6, 0, 6, 4, 0, 4, 4, 1, 8, 4, 6, 2, 4, 3, 4, 0, 4, 3, 7, 8,
       8, 3, 5, 0, 6, 4, 4, 4, 8, 4, 8, 3, 4, 4, 4, 5, 8, 6, 8, 2, 7, 2,
       8, 7, 4, 0, 4, 0, 0, 8, 6, 1, 6, 6, 8, 6, 4, 2, 4, 1, 8, 4, 6, 4,
       6, 4, 6, 9, 4, 0, 3, 4, 5, 0, 7, 5, 4, 7, 8, 4, 8, 0, 8, 8, 3, 2,
       6, 3, 6, 1, 4, 4, 8, 4, 6, 5, 1, 7, 3, 4, 4, 0, 4, 8, 3, 2, 6, 5,
       0, 1, 4, 0, 5, 8, 6, 5, 6, 7, 6, 4, 4, 0, 4, 4, 8, 2, 5, 3, 4, 1,
       8, 0, 6, 8, 6, 7, 7, 4, 8, 2, 4, 4, 7, 4, 0, 0, 8, 0, 6, 5, 6, 4,
       5, 2, 8, 4, 7, 0, 4, 5, 8, 0, 5, 4, 6, 3])